In [21]:
from sentence_transformers import SentenceTransformer
from laser_encoders import LaserEncoderPipeline
from transformers import AutoModel, AutoTokenizer
from accelerate import Accelerator
import torch

In [12]:
accelerator = Accelerator()
device = accelerator.device

In [13]:
sentences = ["This is an example sentence", "Each sentence is converted"]

In [17]:
distiluse_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2').to(device)
laser_encoder = LaserEncoderPipeline(lang="eng_Latn") # 1024
e5_tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
e5_model = AutoModel.from_pretrained('intfloat/multilingual-e5-base').to(device)
e5_model.eval()

2024-12-14 15:13:45,020 | INFO | sentence_transformers.SentenceTransformer | Use pytorch device_name: mps
2024-12-14 15:13:45,020 | INFO | sentence_transformers.SentenceTransformer | Load pretrained SentenceTransformer: sentence-transformers/distiluse-base-multilingual-cased-v2
2024-12-14 15:13:48,651 | INFO | laser_encoders.download_models |  - laser2.spm already downloaded
2024-12-14 15:13:48,694 | INFO | laser_encoders.download_models |  - laser2.pt already downloaded
2024-12-14 15:13:48,694 | INFO | laser_encoders.download_models |  - laser2.spm already downloaded
2024-12-14 15:13:48,694 | INFO | laser_encoders.download_models |  - laser2.cvocab already downloaded


XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=Tru

In [6]:
laser_embedding = laser_encoder.encode_sentences(sentences)

In [18]:
distiluse_embeddings = distiluse_model.encode(sentences)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
print(laser_embedding.shape)

NameError: name 'laser_embedding' is not defined

In [20]:
print(distiluse_embeddings.shape)

(2, 512)


In [ ]:
def e5_embed(docs: list[str]) -> list[list[float]]:
    docs = [f"passage: {d}" for d in docs]
    tokens = e5_tokenizer(docs, padding=True, max_length=512, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        out = e5_model(**tokens)
        last_hidden = out.last_hidden_state.masked_fill(
            ~tokens["attention_mask"][..., None].bool(), 0.0
        )
        doc_embeds = last_hidden.sum(dim=1) / \
            tokens["attention_mask"].sum(dim=1)[..., None]

    return doc_embeds.cpu().numpy()

In [ ]:
e5_embeddings = e5_embed(sentences)

In [ ]:
e5_embeddings.shape